In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import os
print(os.listdir("../input"))


['train.csv', 'merchants.csv', 'sample_submission.csv', 'test.csv', 'historical_transactions.csv', 'Data_Dictionary.xlsx', 'new_merchant_transactions.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
nmt = pd.read_csv('../input/new_merchant_transactions.csv')
mct = pd.read_csv('../input/merchants.csv')


In [ ]:
ht = pd.read_csv('../input/historical_transactions.csv')

In [5]:
data_dict = pd.read_excel('../input/Data_Dictionary.xlsx', sheet_name='train')
data_dict

,train.csv,Unnamed: 1
0,NaN,NaN
1,Columns,Description
2,card_id,Unique card identifier
3,first_active_month,"'YYYY-MM', month of first purchase"
4,feature_1,Anonymized card categorical feature
5,feature_2,Anonymized card categorical feature
6,feature_3,Anonymized card categorical feature
7,target,Loyalty numerical score calculated 2 months af...


In [7]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [4]:
def cal_time(x):
    if isinstance(x['first_active_month'], str):
        d1 = datetime.datetime.strptime(x['first_active_month'],'%Y-%m')
        d2 = datetime.datetime.strptime('2018-12','%Y-%m')
        delta = d2 - d1
    else:
        return float('nan')

    
    return int(delta.days/30)


In [5]:
train['time'] = train.apply(cal_time,axis=1)

In [6]:
train_X = train[['feature_1','feature_2','feature_3','time']]
train_y = train[['target']]
test['time'] = test.apply(cal_time,axis=1)
test_X = test[['feature_1','feature_2','feature_3','time']]
test.loc[11578,'time'] = 22

# historical_transactions

In [17]:
ht.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
authorized_flag         object
card_id                 object
city_id                 int64
category_1              object
installments            int64
category_3              object
merchant_category_id    int64
merchant_id             object
month_lag               int64
purchase_amount         float64
purchase_date           object
category_2              float64
state_id                int64
subsector_id            int64
dtypes: float64(2), int64(6), object(6)
memory usage: 3.0+ GB


In [32]:
ht.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,1,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [37]:
ht_dict = pd.read_excel('../input/Data_Dictionary.xlsx', sheet_name='history')
ht_dict

,historical_transactions.csv,Unnamed: 1
0,NaN,NaN
1,Columns,Description
2,card_id,Card identifier
3,month_lag,month lag to reference date
4,purchase_date,Purchase date
5,authorized_flag,"Y' if approved, 'N' if denied"
6,category_3,anonymized category
7,installments,number of installments of purchase
8,category_1,anonymized category
9,merchant_category_id,Merchant category identifier (anonymized )


In [7]:
ht['authorized_flag'] = ht.authorized_flag.map({'Y':1,'N':0})

In [21]:
ht['authorized_flag'].mean()

0.9135450058482031

In [29]:
card_auth = ht.groupby(['card_id'])['authorized_flag'].mean().sort_values()

In [31]:
card_auth.head()

card_id
C_ID_744d26ea32    0.030488
C_ID_bb8dd137c9    0.037037
C_ID_e56202cd39    0.042105
C_ID_4feacff118    0.050000
C_ID_c27b4f80f7    0.054545
Name: authorized_flag, dtype: float64

In [35]:
ht.groupby(['card_id'])['installments'].mean().head()

card_id
C_ID_00007093c1    1.288591
C_ID_0001238066    1.609756
C_ID_0001506ef0    0.015152
C_ID_0001793786    0.023148
C_ID_000183fdda    1.833333
Name: installments, dtype: float64

In [36]:
ht.groupby(['installments'])['authorized_flag'].mean()

installments
-1      0.885692
 0      0.928032
 1      0.907024
 2      0.884020
 3      0.862254
 4      0.819903
 5      0.809183
 6      0.779732
 7      0.693196
 8      0.692439
 9      0.663721
 10     0.702021
 11     0.660241
 12     0.653694
 999    0.031915
Name: authorized_flag, dtype: float64

In [40]:
ht['subsector_id'].value_counts().sort_values()

 28        124
 11       2009
-1        2252
 40      13424
 24      18134
 23      19724
 14      21825
 5       23171
 13      23388
 26      25140
 35      39269
 4       59102
 9       77760
 10      81954
 3       87853
 38      89005
 2      110596
 32     129516
 17     132915
 31     142945
 22     179252
 12     192067
 41     196118
 25     259053
 39     259206
 30     292335
 20     299301
 36     331696
 18     395679
 8      541510
 15     665634
 1      695263
 7      711977
 21     837612
 16    1651384
 27    1658301
 34    2759431
 29    2813095
 19    3147440
 37    4531755
 33    5594146
Name: subsector_id, dtype: int64

In [44]:
ht.groupby(['subsector_id'])['authorized_flag'].mean().sort_values().head()

subsector_id
 5     0.562945
 8     0.637019
 22    0.698603
-1     0.712256
 26    0.732100
Name: authorized_flag, dtype: float64

In [45]:
ht.groupby(['city_id'])['authorized_flag'].mean().sort_values().head()

city_id
-1      0.776616
 273    0.835484
 92     0.846966
 221    0.859252
 266    0.863066
Name: authorized_flag, dtype: float64

In [50]:
ht['category_1'].value_counts()

N    27028332
Y     2084029
Name: category_1, dtype: int64

In [8]:
ht['category_1'] = ht.category_1.map({'Y':1,'N':0})

In [52]:
ht['category_2'].value_counts()

1.0    15177199
3.0     3911795
5.0     3725915
4.0     2618053
2.0     1026535
Name: category_2, dtype: int64

In [53]:
ht['category_3'].value_counts()

A    15411747
B    11677522
C     1844933
Name: category_3, dtype: int64

In [9]:
ht['category_3'] = ht.category_3.map({'A':0,'B':1,'C':2,'nan':3})

In [3]:
ht.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [13]:
ht['purchase_date'] = pd.to_datetime(ht['purchase_date'])

In [15]:
def feature_eng1(data,prefix):
    data['purchase_month'] = data['purchase_date'].dt.month
    data['month_diff'] = ((datetime.datetime.today() - data['purchase_date']).dt.days)//30
    data['month_diff'] += data['month_lag']
    data['installments'] = data['installments'].astype(int)
    data.loc[:, 'purchase_date'] = pd.DatetimeIndex(data['purchase_date']). \
                                        astype(np.int64) * 1e-9
    data = pd.get_dummies(data, columns=['category_2', 'category_3'])
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean', 'sum'],
        'category_2_2.0': ['mean', 'sum'],
        'category_2_3.0': ['mean', 'sum'],
        'category_2_4.0': ['mean', 'sum'],
        'category_2_5.0': ['mean', 'sum'],
        'category_3_1': ['sum', 'mean'],
        'category_3_2': ['sum', 'mean'],
        'category_3_3': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'month_lag': ['min', 'max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],
        'city_id': ['nunique'],
        'month_diff': ['min', 'max', 'mean']
    }
    agg_trans = data.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (data.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))

    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

    return agg_trans

In [11]:
import gc

In [61]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
gc.collect()
history = feature_eng1(ht, prefix='hist_')
ge.collect()
